In [21]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

# Useful high-level external modules.
import numpy as np
import pandas as pd


from src.distribute_eia923 import *

In [26]:
year = 2020
pudl_db = 'sqlite:///../data/pudl/pudl_data/sqlite/pudl.sqlite'
pudl_engine = sa.create_engine(pudl_db)

start_date='2020-01-01'
end_date='2020-12-31'

plants_ba = plants_eia860(pudl_engine, start_date=start_date, end_date=end_date)[['plant_id_eia','balancing_authority_code_eia','state']]

plants_ba

,plant_id_eia,balancing_authority_code_eia,state
0,1,None,AK
1,2,SOCO,AL
2,3,SOCO,AL
3,4,SOCO,AL
5,7,SOCO,AL
...,...,...,...
14435,64876,ISNE,MA
14436,64877,CISO,CA
14437,64878,CISO,CA
14438,64879,CISO,CA


In [2]:
gen_allocated = allocate_gen_fuel_by_gen(year=2020)

c:\Users\Greg\GitHub\hourly-egrid\notebooks\../../hourly-egrid\src\distribute_eia923.py:797: UserWarning: Ooopsies. You got 249 records where the 'frac' column isn't adding up to 1 for each 'IDX_PM_FUEL' group. Check 'calc_allocation_fraction()'
        plant_id_eia prime_mover_code energy_source_code report_date  frac  \
3413             377               CA                 NG  2020-04-01   2.0   
3414             377               CT                 NG  2020-04-01   2.0   
3441             377               CA                 NG  2020-11-01   2.0   
3442             377               CT                 NG  2020-11-01   2.0   
3445             377               CA                 NG  2020-12-01   2.0   
...              ...              ...                ...         ...   ...   
104483         58207               CA                 NG  2020-03-01   4.0   
104487         58207               CA                 NG  2020-04-01   4.0   
104495         58207               CA               

In [6]:
gen_allocated[gen_allocated['plant_id_eia'] == 3].sum()

C:\Users\Greg\AppData\Local\Temp/ipykernel_9504/3358685187.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  gen_allocated[gen_allocated['plant_id_eia'] == 3].sum()


plant_id_eia                                                         360
plant_id_pudl                                                       3840
plant_name_eia         BarryBarryBarryBarryBarryBarryBarryBarryBarryB...
utility_id_eia                                                   23400.0
utility_id_pudl                                                     2160
utility_name_eia       Alabama Power CoAlabama Power CoAlabama Power ...
generator_id           1245A1CTA1CT2A1STA2C1A2C2A2ST1245A1CTA1CT2A1ST...
fuel_consumed_mmbtu                                           82061242.0
net_generation_mwh                                          10499146.011
unit_id_pudl                                                       612.0
dtype: object

In [7]:
import sqlalchemy as sa
# specify the year for analysis
year = 2020

# specify the relative path to the sqllite database, and create an sqalchemy engine
pudl_db = 'sqlite:///../data/pudl/pudl_data/sqlite/pudl.sqlite'
pudl_engine = sa.create_engine(pudl_db)

# load the generation fuel data for the year
generation_eia923 = pd.read_sql(f"SELECT * FROM generation_eia923 WHERE report_date >= '{year}-01-01' AND report_date <= '{year}-12-01'", pudl_engine)
generation_fuel_eia923 = pd.read_sql(f"SELECT * FROM generation_fuel_eia923 WHERE report_date >= '{year}-01-01' AND report_date <= '{year}-12-01'", pudl_engine)

In [10]:
generation_fuel_eia923[(generation_fuel_eia923['plant_id_eia'] == 3)].sum()

plant_id_eia                                                                         165
report_date                            2020-02-012020-03-012020-04-012020-05-012020-0...
energy_source_code                     NGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGBI...
fuel_type_code_pudl                    gasgasgasgasgasgasgasgasgasgasgasgasgasgasgasg...
fuel_type_code_aer                     NGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGNGCO...
prime_mover_code                       CACACACACACACACACACACACTCTCTCTCTCTCTCTCTCTCTST...
fuel_consumed_units                                                           56086082.0
fuel_consumed_for_electricity_units                                           56086082.0
fuel_mmbtu_per_unit                                                              258.481
fuel_consumed_mmbtu                                                           82061242.0
fuel_consumed_for_electricity_mmbtu                                           82061242.0
net_generation_mwh   

In [10]:
generation_fuel_eia923[(generation_fuel_eia923['plant_id_eia'] == 404) & (generation_fuel_eia923['report_date'] == '2020-01-01')]

,plant_id_eia,report_date,energy_source_code,fuel_type_code_pudl,fuel_type_code_aer,prime_mover_code,fuel_consumed_units,fuel_consumed_for_electricity_units,fuel_mmbtu_per_unit,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,net_generation_mwh
4098,404,2020-01-01,NG,gas,NG,CA,0.0,0.0,0.000,0.0,0.0,-3.0
4110,404,2020-01-01,OG,gas,OOG,CA,0.0,0.0,0.000,0.0,0.0,0.0
4122,404,2020-01-01,NG,gas,NG,CT,0.0,0.0,0.000,0.0,0.0,0.0
4134,404,2020-01-01,OG,gas,OOG,CT,0.0,0.0,0.000,0.0,0.0,0.0
4146,404,2020-01-01,NG,gas,NG,GT,158421.0,158421.0,1.044,165392.0,165392.0,17618.0
4158,404,2020-01-01,OBG,waste,ORW,GT,0.0,0.0,0.000,0.0,0.0,0.0
4170,404,2020-01-01,NG,gas,NG,ST,26241.0,26241.0,1.044,27396.0,27396.0,2288.0
4182,404,2020-01-01,OBG,waste,ORW,ST,0.0,0.0,0.000,0.0,0.0,0.0


In [11]:
import src.load_data as load_data
import sqlalchemy as sa

year = 2020

IDX_GENS = ["plant_id_eia", "generator_id", "report_date"]
"""Id columns for generators."""

IDX_PM_FUEL = ["plant_id_eia", "prime_mover_code", "energy_source_code", "report_date"]
"""Id columns for plant, prime mover & fuel type records."""

IDX_FUEL = ["report_date", "plant_id_eia", "energy_source_code"]

DATA_COLS = ["net_generation_mwh", "fuel_consumed_mmbtu"]
"""Data columns from generation_fuel_eia923 that are being allocated."""

pudl_db = 'sqlite:///../data/pudl/pudl_data/sqlite/pudl.sqlite'
pudl_engine = sa.create_engine(pudl_db)

# specify the date filter for retrieving data
year_filter = f"report_date >= '{year}-01-01' AND report_date <= '{year}-12-01'"

# extract all of the tables from pudl_out early in the process and select
# only the columns we need. this is for speed and clarity.
gf = load_data.load_pudl_table(f"SELECT * FROM generation_fuel_eia923 WHERE {year_filter}").loc[
    :, IDX_PM_FUEL + ["net_generation_mwh", "fuel_consumed_mmbtu"]
].pipe(apply_dtype)
gen = (
    load_data.load_pudl_table(f"SELECT * FROM generation_eia923 WHERE {year_filter}").loc[:, IDX_GENS + ["net_generation_mwh"]]
    # removes 4 records with NaN generator_id as of pudl v0.5
    .dropna(subset=IDX_GENS)
).pipe(apply_dtype)
gens = load_data.load_pudl_table(f"SELECT * FROM generators_eia860 WHERE {year_filter}").loc[
    :,
    IDX_GENS
    + [
        "capacity_mw",
        "operational_status",
        "retirement_date",
    ]
    + list(load_data.load_pudl_table(f"SELECT * FROM generators_eia860 WHERE {year_filter}").filter(like="energy_source_code")),
]
# add the prime mover code to the gens df from generators entity
gens = gens.merge(load_data.load_pudl_table("generators_entity_eia").loc[:,["plant_id_eia", "generator_id","prime_mover_code"]],
                    how='left', 
                    on=["plant_id_eia", "generator_id"]).pipe(apply_dtype)

In [30]:
gf[gf['net_generation_mwh'] < 0]

,plant_id_eia,prime_mover_code,energy_source_code,report_date,net_generation_mwh,fuel_consumed_mmbtu
46,3,ST,BIT,2020-02-01,-152.343,1344.0
57,3,ST,NG,2020-02-01,-3391.657,29922.0
95,7,ST,NG,2020-05-01,-397.000,0.0
96,7,ST,NG,2020-06-01,-335.000,0.0
97,7,ST,NG,2020-07-01,-341.000,0.0
...,...,...,...,...,...,...
170950,64295,BA,MWH,2020-12-01,-20.892,0.0
170951,64296,BA,MWH,2020-11-01,-15.424,0.0
170952,64296,BA,MWH,2020-12-01,-14.576,0.0
170953,64297,BA,MWH,2020-12-01,-4.000,0.0


In [ ]:
# duplicate the entries for each month
gens_month = gens.copy()

month = 2
while month <= 12:
    # add one month to the copied data each iteration
    gens_month['report_date'] = gens_month['report_date'] + pd.DateOffset(months=1)
    # concat this data to the gens file
    gens = pd.concat([gens, gens_month], axis = 0)
    month += 1

